In [1]:
#!pip install seaborn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import time

#!pip install geopy
#!pip install googletrans==3.1.0a0
from googletrans import Translator
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import combinations
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import WordPunctTokenizer

#import nltk
#nltk.download('stopwords')
from nltk.corpus import stopwords
#!pip install sentence_transformers
from sentence_transformers import SentenceTransformer, util
import pycountry_convert as pc

In [11]:
train = pd.read_csv('conversion_train_etc_duplicate.csv')
submission = pd.read_csv('conversion_sub_etc_duplicate.csv')

In [4]:
'''
train = pd.read_csv('train.csv')
submission = pd.read_csv('submission.csv')

#### 중복행 제거
# 범주형 변수 -> Etc, etc와 같이 정제되어있지 않은 범주형 변수를 간단히 정제 : 같은 etc로 분류되어있지만 다른 데이터라고 되어있는 것을 막기 위함
label_columns = [
    "customer_country",
    "business_subarea",
    "business_area",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "product_subcategory",
    "product_modelname",
    "customer_country.1",
    "customer_position",
    "response_corporate",
    "expected_timeline",
]

# 특수문자 제거, 소문자 변환, 공백 제거
def regenerate_func(str_c):
    new_str = re.sub(r"[.,~!@#$%^&*\(\)_+=\-~|\\\"\';:\?/\<\>\[\]\{\}]", " ", str_c) # 특수문자
    new_str = ''.join(new_str.split())
    new_str = new_str.strip() # 양옆 공백제거
    new_str = new_str.lower() # 소문자 변환
    
    return new_str

# 범주형 변수인 경우 -> 특수문제 제거, 소문자 변환, 공백 제거 => ex) Korea와 korea는 동일, N/A와 NA는 동일
df_copy = train.copy()
df_copy[label_columns] = df_copy[label_columns].apply(lambda x: [regenerate_func(i) if not pd.isna(i) else i for i in x])  # regenerate function 적용

# 중복 데이터 확인
duplication = df_copy.duplicated().values
duplication = [int(not i) for i in duplication] # False인 경우 1, True인 경우 0으로 변환 -> False인 경우는 중복 행 중 첫번째를 의미

# 중복 갯수 산출
idx = 0
for i in range(len(train)):
    if len(train)-1 == i:  # 마지막 행은 중복행이자 첫번째 행일 수 없음. 즉, 1 이면 그대로 1, 0이면 그대로 0으로 출력
        continue
    
    if duplication[i] == 1:  # 1이면 첫번째 행이므로 그대로 출력, idx를 i로 변환해 i번째 행이 첫번째 행임을 나타냄
        idx = i
    else:
        duplication[idx] += 1  # i번째 행이 0이면 idx로 저장해둔 즉, 중복행 중 첫번째 행이 +1 -> 갯수를 셀 수 있음

# 중복행 갯수 columns 추가
train["duplication"] = duplication
train = train[train.duplication != 0]

def translate_func(text, attempt = 1, max_attempt = 5):
    translator = Translator()
    try:
        return translator.translate(text, dest = "en").text  # 번역
    except:          # 에러의 경우
        if attempt <= max_attempt:   # 5번 이하로 다시 시도
            return translate_func(text, attempt = attempt + 1)
        raise
        
### 국가만 추출 ------------------ 240208 추가(박지호)
# 규칙에 맞는 국가
def country_func(country_str):
    country_split = country_str.split("//")
    if len(country_str.split('//')) != 1 and country_str.split("//")[-1].strip() != '':
        country_str = country_str.split("//")[-1]
    elif len(country_str.split("/")) != 1 and country_str.split("/")[-1].strip() != "":  # 주소/주소/국가인 경우
        country_str = country_str.split("/")[-1]
    return country_str

# 규칙에 맞지 않는 국가 - 약 40분
def geo_country_func(country_str, attempt = 1, max_attempt = 5):
    time.sleep(1)  # 딜레이 -> geolocoder가 1초에 1번만 실행해야함
    
    global n      # 몇 번째 행인지
    global start_t   # 얼마나 걸리는 지
    if len(country_str.split("/")) == 4 and country_str.split("/")[-1].strip() != "":
        country_str = country_str.split("/")[-1]
    elif len(country_str.split("/")) == 3 and country_str.split("/")[1].strip() != "":
        country_str = country_str.split("/")[1]
    elif country_str.split(",")[-1].strip() != "":
        country_str = country_str.split(",")[-1]
    
    try:
        n += 1
        geo = geolocoder.geocode(country_str) # 주소 검색
        if pd.isna(geo):         # 주소가 없는 경우
            country_str = geo    # 공백
        else:
            country_str = geo[0].split(",")[-1].strip()  # 주소가 있는 경우 추출
        
        # 현재 진행상황 및 시간 출력
        if n % 100 ==0:
            print("------------------ Step {} : {} -------------------".format(n, time.time() - start_t))
            start_t = time.time()
        
        return country_str
    
    except GeocoderTimedOut:  # error인 경우
        if attempt <= max_attempt:   # 5번 이하로 재시도
            return geo_country_func(country_str, attempt=attempt+1)
        raise

# 규칙에 맞는 국가 추출
train.customer_country = [country_func(i) if not pd.isna(i) else i for i in train.customer_country]
submission.customer_country = [country_func(i) if not pd.isna(i) else i for i in submission.customer_country]

# 규칙에 맞지 않는 국가 추출
n = 0
geolocoder = Nominatim(user_agent = 'South Korea', timeout=None)
country_train = list(set(train.customer_country)) # 국가 추출 전 raw data
conversion_train = [geo_country_func(i) if not pd.isna(i) else i for i in country_train] # 국가 추출 후 data

n = 0
country_sub = list(set(submission.customer_country)) # 국가 추출 전 raw data
conversion_sub = [geo_country_func(i) if not pd.isna(i) else i for i in country_sub] # 국가 추출 후 data

# 번역
conversion_trans_train = [i.split("/")[-1] if not pd.isna(i) else i for i in conversion_train]  # 규칙에 맞지 않는 국가에서 특수문자가 아직 포함되어있는 경우 국가만 추출
conversion_trans_train = [translate_func(i) if not pd.isna(i) else i for i in conversion_trans_train] # 번역
conversion_trans_sub = [i.split("/")[-1] if not pd.isna(i) else i for i in conversion_sub]  # 규칙에 맞지 않는 국가에서 특수문자가 아직 포함되어있는 경우 국가만 추출
conversion_trans_sub = [translate_func(i) if not pd.isna(i) else i for i in conversion_trans_sub] # 번역

# 카테고리명 변경
conversion_df_train = pd.DataFrame([country_train, conversion_trans_train]).T  # DataFrame 생성
conversion_df_train.columns = ["country", "conversion"]            # colunm명 생성
conversion_dict_train = conversion_df_train.set_index("country").T.to_dict()   # dict형으로 변환
conversion_re_train = [conversion_dict_train[i]["conversion"] if not pd.isna(i) else i for i in train.customer_country] # country 기준 conversion으로 변경

conversion_df_sub = pd.DataFrame([country_sub, conversion_trans_sub]).T  # DataFrame 생성
conversion_df_sub.columns = ["country", "conversion"]            # colunm명 생성
conversion_dict_sub = conversion_df_sub.set_index("country").T.to_dict()   # dict형으로 변환
conversion_re_sub = [conversion_dict_sub[i]["conversion"] if not pd.isna(i) else i for i in submission.customer_country] # country 기준 conversion으로 변경

# 국가를 찾을 수 없는 카테고리는 원래의 카테고리명으로 - 결측치 그대로
conversion_na_train = [train.customer_country.iloc[i] if pd.isna(conversion_re_train[i]) else conversion_re_train[i] for i in range(len(conversion_re_train))]
conversion_na_sub = [submission.customer_country.iloc[i] if pd.isna(conversion_re_sub[i]) else conversion_re_sub[i] for i in range(len(conversion_re_sub))]

# 국가를 찾을 수 없는 카테고리는 etc로 변경 - 결측치 0
conversion_etc_train = ["etc" if pd.isna(conversion_re_train[i]) else conversion_re_train[i] for i in range(len(conversion_re_train))]
conversion_etc_sub = ["etc" if pd.isna(conversion_re_sub[i]) else conversion_re_sub[i] for i in range(len(conversion_re_sub))]
'''

In [12]:
train = train.drop("duplication", axis = 1)

In [13]:
############################## customer_country ##############################################
###### => customer_continent 변수 생성 : 대륙으로 나누는 변수
country_names = {
    'The United Arab Emirates': 'United Arab Emirates',
    'Democratic Republic of Congo': 'Democratic Republic of the Congo',
    'Papua Niugini': 'Papua New Guinea',
    'The Savior': 'El Salvador',
    'Sri Lanka Sri Lanka': 'Sri Lanka',
    'korea': 'South Korea',
    'Deutschland': 'Germany',
    'Algérie ⵍⵣⵣⴰⵢⴻⵔ Algeria': 'Algeria',
    'Italia': 'Italy',
    'The Netherlands': 'Netherlands',
    'Viti': 'Fiji',
    'Cyprus - Kıbrıs': 'Cyprus',
    'Dhvehiraje': 'Serbia',
    'Maroc ⵍⵎⵖⵔⵉⴱ Morocco': 'Morocco',
    'Swisher': 'Switzerland',
    'The Bahamas': 'Bahamas',
    'Mongolia ᠮᠤᠩᠭᠤᠯ ᠤᠯᠤᠰ': 'Mongolia',
    'Soomaaliya Somalia': 'Somalia',
    'Maurice': 'Mauritius',
    'Hellas': 'Greece',
    'Benign': 'Benin',
    'in Swat': 'Pakistan',
    'Kosovo': 'Italy',                      #kosovo를 인식하지 못해서 다른 유럽 국가로 대체함
    'Djibouti Djibouti': 'Djibouti'
}

# 국가명 재변환을 위한 mapping 함수
def map_country(country):
    if country in country_names.keys():
        return country_names[country]
    else:
        return country
    
#country_names mapping
train['customer_country'] = train['customer_country'].apply(map_country)
submission['customer_country'] = submission['customer_country'].apply(map_country)

#나라를 대륙 명으로 변환하는 함수 
def country_to_continent(country_name):
    try:
        country_alpha2 = pc.country_name_to_country_alpha2(country_name)
        country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
        country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
        return country_continent_name
    except Exception as e:
        if country_name == 'etc':
            return 'etc'
        return 'error'

train['customer_continent'] = train['customer_country'].apply(country_to_continent) 
submission['customer_continent'] = submission['customer_country'].apply(country_to_continent) 

############################## business_unit ##############################################
#### => CM, Solution을 Etc로 변환
train['business_unit'].replace({'CM':'etc','Solution':'etc'},inplace=True)
submission['business_unit'].replace({'CM':'etc','Solution':'etc'},inplace=True)

############################## com_reg_ver_win_rate ##############################################
###### => 결측치를 0으로 처리
train['com_reg_ver_win_rate'].fillna(0,inplace=True)
submission['com_reg_ver_win_rate'].fillna(0,inplace=True)

############################## customer_idx ##############################################
#### => mean_customer_idx 생성 : 빈도수가 5 이하인 회사는 평균값으로 처리
#### => 기존의 customer_idx는 삭제

# 회사의 카테고리가 너무 많고, 특정 회사의 값이 너무 크고, 다른 회사는 데이터 수가 적고 값이 큰 영향을 주지 않아 학습을 해친다고 생각해 삭제하기로 합의
#삭제
train['customer_idx'] = train['customer_idx'].astype(str)
submission['customer_idx'] = submission['customer_idx'].astype(str)

# customer_idx_cnt 컬럼 생성
train['customer_idx_cnt'] = train.groupby(['customer_idx'])['customer_idx'].transform('count')

#빈도 수 (customer_idx_cnt)가 5 이하인 데이터는 customer_idx를 etc로 통일
n = 5
for i in range(1,n+1):
    train.loc[train['customer_idx_cnt'] == i,'customer_idx'] = 'etc'

# customer_idx 별 평균 영업 성공률 계산한 mean_customer_idx 추가
train['mean_customer_idx'] = train.groupby('customer_idx')['is_converted'].transform('mean')

# etc면 전체 영업 성공률로 변환
converted_mean = sum(train.is_converted)/len(train)
train.mean_customer_idx = [converted_mean if train.customer_idx.iloc[i] == "etc" else train.mean_customer_idx.iloc[i] for i in range(len(train))]

mean_customer_idx_unique = train.drop_duplicates(subset=['customer_idx'])[['customer_idx', 'mean_customer_idx']]
submission = submission.merge(mean_customer_idx_unique, on='customer_idx', how='left')

#submission에서 매칭되지 않은(etc에 속한 customer_idx 또는 submission에만 존재하는 customer_idx) mean_customer_idx의 값은 etc에 해당하는 값으로 대체
submission['mean_customer_idx'].fillna(train[train['customer_idx']=='etc']['mean_customer_idx'][0],inplace= True)

############################## customer_type ##############################################
#### => 챗지피티를 이용해 카테고리 그룹화, 결측값은 Other
customer_types = {
    'End-Customer': 'End Customer',
    'End-user': 'End Customer',
    'Corporate': 'End Customer',
    'End Customer': 'End Customer',
    'Homeowner': 'End Customer',
    'Home Owner': 'End Customer',
    'Commercial end-user': 'End Customer',
    'Specifier/ Influencer': 'Specifier/Influencer',
    'Specifier / Influencer': 'Specifier/Influencer',
    'Architect/Consultant': 'Specifier/Influencer',
    'Manager / Director': 'Specifier/Influencer',
    'Consultant': 'Specifier/Influencer',
    'Administrator': 'Specifier/Influencer',
    'Developer': 'Engineer',
    'Engineer': 'Engineer',
    'HVAC Engineer': 'Engineer',
    'System Integrator': 'Engineer',
    'Installer': 'Engineer',
    'Installer/Contractor': 'Engineer',
    'Technician': 'Engineer',
    'Technical Assistant': 'Engineer',
    'Technician': 'Engineer',
    'Service Partner': 'Channel Partner',
    'Channel Partner': 'Channel Partner',
    'Solution Eco-Partner': 'Channel Partner',
    'Distributor': 'Channel Partner',
    'Reseller': 'Channel Partner',
    'Dealer/Distributor': 'Channel Partner',
    'Software/Solution Provider': 'Channel Partner',
    'Software / Solution Provider': 'Channel Partner',
    'Dealer': 'Channel Partner',
    'Others': 'Other',
    'Etc.': 'Other',
    'Interior Designer': 'Other'
}

# mapping 함수 정의
def map_customer_type(type):
    if type in customer_types.keys():
        return customer_types[type]
    else:
        return type

# Mapping
train['customer_type'] = train['customer_type'].map(map_customer_type)
submission['customer_type'] = submission['customer_type'].map(map_customer_type)

# 결측값은 Other로 변환
train['customer_type'].fillna('Other',inplace=True)
submission['customer_type'].fillna('Other',inplace=True)

############################## customer_type ##############################################
#### => 결측값을 0으로 채움
#### => historical_existing_cnt_isna 변수 생성 : 영업 전환 성공 여부
idx = train[train.historical_existing_cnt.isna()].index
train.loc[idx, "historical_existing_cnt"] = 0

idx = submission[submission.historical_existing_cnt.isna()].index
submission.loc[idx, "historical_existing_cnt"] = 0

# 영업 전환 시도 변수 생성 -> 전환 한 적이 있으면 1, 아니면 0
train["historical_existing_cnt_isna"] = [0 if i == 0 else 1 for i in train.historical_existing_cnt]
submission["historical_existing_cnt_isna"] = [0 if i == 0 else 1 for i in submission.historical_existing_cnt]

############################## id_strategic_ver, it_strategic_ver, idit_strategic_ver ##############################################
#### => 결측치를 0으로 변환
train['id_strategic_ver'].fillna(0,inplace=True)
submission['id_strategic_ver'].fillna(0,inplace=True)

train['it_strategic_ver'].fillna(0,inplace=True)
submission['it_strategic_ver'].fillna(0,inplace=True)

train['idit_strategic_ver'].fillna(0,inplace=True)
submission['idit_strategic_ver'].fillna(0,inplace=True)

############################## inquiry_type ##############################################
#### => inquiry_type_isna 변수 생성 : 요청사항 유무
train["inquiry_type_isna"] = [0 if pd.isna(i) else 1 for i in train.inquiry_type]
submission["inquiry_type_isna"] = [0 if pd.isna(i) else 1 for i in submission.inquiry_type]

############################## product_modelname ##############################################
#### => 정확한 모델명이 있는지 여부로 변환
def is_specific_model_name(entry):
    """
    Determines if an entry is likely a specific model name based on predefined rules.
    
    Args:
    - entry (str): The string to be evaluated.
    
    Returns:
    - int: 1 if the entry is likely a specific model name, otherwise 0.
    """
    
    # Check if entry is not a string (e.g., NaN or None)
    if not isinstance(entry, str):
        return 0  # Assume non-strings are not specific model names
    
    # Rule 1: Exclude obvious non-model entries based on common keywords
    common_non_model_keywords = ["service", "need", "inquiry", "meeting", "only", "want", "all"]
    if any(keyword in entry.lower() for keyword in common_non_model_keywords):
        return 0
    
    # Rule 2: Check for alphanumeric patterns typical of model names
    if re.search(r'\d', entry) and re.search(r'[A-Z]', entry):
        # Rule 3: Check for presence of specific special characters
        if re.search(r'[-()_/]', entry) or re.search(r'\d{2,}', entry):
            return 1
    
    # Default to non-specific if no rules matched
    return 0

train['product_modelname'] = train['product_modelname'].apply(is_specific_model_name)
submission['product_modelname'] = submission['product_modelname'].apply(is_specific_model_name)

############################## customer_position ##############################################
#### => customer_position_power 변수 생성 : 영향력이 있는 직책인지 여부
def categorize_position(position):
    # High decision-making power
    high_power_positions = ['ceo/founder', 'c-level executive', 'vice president', 'director', 'partner', 'manager', 'business unit director', 'chief executive officer', 'chairman', 'president', 'decision maker', 'decision-maker']
    # Normalize the position to handle variations in case and spacing
    position_normalized = position.lower().replace(" ", "")
    for high_power_position in high_power_positions:
        if high_power_position.replace(" ", "") in position_normalized:
            return 1
    # Default to no decision-making power
    return 0

# Apply the categorization function to create a new column
train['customer_position_power'] = train['customer_position'].apply(categorize_position)
submission['customer_position_power'] = submission['customer_position'].apply(categorize_position)

############################## response_corporate ##############################################
#### => response_region 생성 : 지사가 있는 대륙 변수
#### => corporate_converted_ratio 생성 : 지사별 영업 성공률
response_corporate_region = {
    'LGEPH': 'Asia',
    'LGEIL': 'Middle East',
    'LGEAF': 'Africa',
    'LGESJ': 'Europe',
    'LGESL': 'Europe',
    'LGESP': 'Europe',
    'LGEGF': 'Africa',
    'LGESA': 'Middle East',
    'LGEUS': 'North America',
    'LGECB': 'Latin America',
    'LGEMS': 'Asia',
    'LGEEG': 'Middle East',
    'LGEEF': 'Africa',
    'LGEAP': 'Oceania',
    'LGEIN': 'Asia',
    'LGEUK': 'Europe',
    'LGEKR': 'Asia',
    'LGEPS': 'Latin America',
    'LGECI': 'Africa',
    'LGECL': 'Latin America',
    'LGETK': 'Europe',
    'LGELF': 'Europe',
    'LGEPT': 'Europe',
    'LGEPR': 'Latin America',
    'LGEDG': 'Latin America',
    'LGERO': 'Europe',
    'LGEMK': 'Europe',
    'LGEPL': 'Europe',
    'LGECZ': 'Europe',
    'LGEES': 'Europe',
    'LGEAR': 'Middle East',
    'LGEHK': 'Asia',
    'LGEML': 'Africa',
    'LGEJP': 'Asia',
    'LGEHS': 'Latin America',
    'LGEAS': 'Asia',
    'LGEYK': 'Middle East',
    'LGEIS': 'Europe',
    'LGEBN': 'Africa',
    'LGEFS': 'Europe',
    'LGESW': 'Europe',
    'LGEMC': 'Europe',
    'LGEAG': 'Africa',
    'LGEEB': 'Middle East',
    'LGETH': 'Africa',
    'LGEVH': 'Asia',
    'LGECH': 'Europe',
    'LGELA': 'Asia',
    'LGETT': 'Europe',
    'LGERA': 'Africa',
    'LGEUR': 'Europe',
    'LGEIR': 'Europe',
    'LGEBT': 'Asia'
}

#region_continents 생성
train['response_region'] = train['response_corporate'].apply(lambda x : response_corporate_region[x])
submission['response_region'] = submission['response_corporate'].apply(lambda x : response_corporate_region[x])

# 지사별 영업성공률 파생변수 생성
table = train.value_counts(["response_corporate","is_converted"]).reset_index().pivot(index = "is_converted", columns = "response_corporate", values = 0).T
table["Total"] = table.sum(axis = 1)
table["True_per"] = table[True]/table["Total"]
table = table.fillna(0)
table = table.reset_index()[["response_corporate", "True_per"]].set_index("response_corporate").T.to_dict()
train['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in train.response_corporate]
submission['corporate_converted_ratio'] = [table[i]["True_per"] if i in table.keys() else converted_mean for i in submission.response_corporate]


#특수문자와 공백을 모두 제거
def remove_special_characters(text):
    if isinstance(text, str):
        # 정규표현식을 사용하여 공백과 특수문자를 제거하고 모든 공백을 없앰
        cleaned_text = re.sub(r'[^a-zA-Z0-9가-힣]', '', text)
        return cleaned_text.strip()  # 양쪽 공백 제거 후 반환
    else:
        return text
############################## response_corporate ##############################################
#### -> 총 6가지의 카테고리로 변환
expected_timeline_class = ['lessthan3months', 'morethanayear','6months9months', '3months6months', '9months1year']
label = {'lessthan3months' : 1, '3months6months' : 2,'6months9months' : 3, '9months1year' : 4, 'morethanayear' : 5}

#공백과 특수문자 제거
train['expected_timeline'] = train['expected_timeline'].apply(remove_special_characters)
submission['expected_timeline'] = submission['expected_timeline'].apply(remove_special_characters)

#label encoding
train['expected_timeline'] = train['expected_timeline'].apply(lambda x: label[x] if x in label else 0)
submission['expected_timeline'] = submission['expected_timeline'].apply(lambda x: label[x] if x in label else 0)


############################## ver_win_rate_x, ver_win_ratio_per_bu ##############################################
#### => 결측치를 0으로 채움
train.ver_win_rate_x = train.ver_win_rate_x.fillna(0)
submission.ver_win_rate_x = submission.ver_win_rate_x.fillna(0)

train.ver_win_ratio_per_bu = train.ver_win_ratio_per_bu.fillna(0)
submission.ver_win_ratio_per_bu = submission.ver_win_ratio_per_bu.fillna(0)

############################## ver_win_rate_x, ver_win_ratio_per_bu ##############################################
#### => mean_lead_owner 생성 : 기업별 영업 성공률(빈도수가 5 이하면 전체 영업 성공률로 변환)
# lead_owner_cnt 컬럼 생성
train['lead_owner_cnt'] = train.groupby(['lead_owner'])['lead_owner'].transform('count')

#빈도 수 (lead_owner_cnt)가 5 이하인 데이터는 lead_owner를 etc로 통일
n = 5
for i in range(1,n+1):
    train.loc[train['lead_owner_cnt'] == i,'lead_owner'] = 'etc'

# lead_owner 별 평균 영업 성공률 계산한 mean_lead_owner 추가
train['mean_lead_owner'] = train.groupby('lead_owner')['is_converted'].transform('mean')

train.mean_lead_owner = [converted_mean if train.lead_owner.iloc[i] == "etc" else train.mean_lead_owner.iloc[i] for i in range(len(train))]
mean_customer_idx_unique = train.drop_duplicates(subset=['lead_owner'])[['lead_owner', 'mean_lead_owner']]
submission = submission.merge(mean_customer_idx_unique,on='lead_owner',how='left')

#submission에서 매칭되지 않은(etc에 속한 lead_owner 또는 submission에만 존재하는 lead_owner) mean_customer_idx의 값은 etc에 해당하는 값으로 대체
submission['mean_lead_owner'].fillna(train[train['lead_owner']=='etc']['mean_lead_owner'].iloc[0],inplace= True)

In [14]:
train_dropcol = ["product_category", "product_subcategory", "customer_country.1", "customer_position", "customer_idx_cnt", "customer_idx",
                 "business_area", "inquiry_type", "business_subarea", "lead_owner", "lead_owner_cnt"]
sub_dropcol = ["product_category", "product_subcategory", "customer_country.1", "customer_position", "customer_idx", "business_area", 
               "inquiry_type", "business_subarea", "lead_owner"]

train.drop(train_dropcol, axis = 1, inplace=True)
submission.drop(sub_dropcol, axis = 1, inplace=True)

In [ ]:
train.to_csv("complete_train_duplication.csv", index = False)
submission.to_csv("complete_sub_duplication.csv", index = False)